In [2]:
# 문서 로드
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/SPRi AI Brief_10월호_산업동향_F.pdf")
docs = loader.load()

In [3]:
# 문서 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
split_documents = text_splitter.split_documents(docs)

In [4]:
# 임베딩
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [8]:
# DB 생성 및 저장
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

for doc in vectorstore.similarity_search("AI 규제법"):
    print(doc.page_content)

∙AI를 실행하거나 훈련하는 물리적 인프라를 보유한 국가는 규정 준수를 강제할 수 있으나, AI 인프라 
관할권이 없는 국가는 규제 권한이 없어 인프라를 보유한 국가의 거버넌스에 종속될 위험
☞ 출처: Center for Open Science, Compute North vs. Compute South: The Uneven Possibilities of Compute-based AI 
Governance Around the Globe, 2024.08.22.
사례를 공유하고 국가 및 지역적 접근방식 간의 조정 및 상호운용성을 개선
∙(AI 표준 교류 체계 마련) 다양한 표준 개발 기구, 기술 기업, 시민사회 대표들 간 AI 표준 교류를 위한 
포괄적인 체계를 마련해 개방적이고 상호 운용이 가능하며 신뢰할 수 있는 AI 생태계 구축
∙(AI 역량 개발 네트워크 구축) 공공 및 민간 부문의 모든 사람이 AI를 책임 있고 윤리적으로 사용할 수 
있도록 포괄적이고 공평한 AI 역량 개발 네트워크를 구축
각각 9만 개와 5만 개의 일자리가 감소 예상
∙AI와 같은 복잡한 기술의 구현과 유지 관리를 위해 40만 개의 기술 일자리가 생겨날 전망으로, 
컴퓨터와 정보시스템 관리, 개발 및 데이터 엔지니어링 분야가 특히 유망
n AI는 새로운 일자리를 창출하는 동시에 일상 업무의 자동화를 통해 기존 근로자의 생산성을 개선할 
전망으로, AI는 영국 내 정규직 288만 명에 상당하는 작업을 수행 가능
∙시스템 관리자와 같은 기술 분야의 직원은 AI 활용으로 주당 최대 12.6시간을 절약하여 남는 시간을 
더욱 복잡한 작업에 투입 가능
첫 국제조약으로, 서명국은 조약 이행을 위한 자국 내 입법이나 행정 조치 채택 필요
∙조약은 AI 시스템의 수명주기 전반에서 인간의 존엄성과 개인의 자율성 존중, 투명성, 책임성, 평등과 
차별금지, 개인정보보호와 같은 기본원칙을 제시
∙AI 시스템 수명주기 내 활동으로 인한 피해에 대하여 효과적인 구제 수단을 보장하고 AI 시스템이 
인권·민주주의

In [7]:
# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [15]:
# 프롬프트 생성
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question.
    Use a very kind and gentle tone like a kindergarten teacher talking to a child.
    Speak in a warm and friendly way.
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

    #Context: 
    {context}

    #Question:
    {question}

    #Answer:
    """
)

In [9]:
# LLM 모델 생성
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [16]:
# Chain 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
# Chain 실행
question = "캘리포니아주 주지사가 AI 규제법의 입법을 거부한 이유는 무엇인가요?"
response = chain.invoke(question)
print(response)

캘리포니아 주지사인 개빈 뉴섬은 AI 규제법 「SB1047」에 거부권을 행사했어요. 그 이유는 이 법안이 크고 비싼 AI 모델 규제에만 집중하고 있어서, 기술의 위협에 대응하는 최선의 접근방식이 아니라고 생각했기 때문이에요. 뉴섬 주지사는 이 법안이 AI 시스템이 위험한 상황에 사용되는지, 중요한 의사 결정에 관여하거나 민감한 데이터를 사용하는지를 충분히 고려하지 않는다고 지적했어요. 그래서 법안이 대중을 AI 기술의 위협에서 보호하는 데 충분하지 않다고 판단한 거예요.
